In [4]:
!pip install tensorflow_text

     |████████████████████████████████| 4.6 MB 5.0 MB/s 
     |██████████████████████████████  | 479.8 MB 13.6 MB/s eta 0:00:03
ERROR: Operation cancelled by user


In [5]:
import tensorflow as tf
import tensorflow_text as tf_text
import subprocess
import os
import pandas as pd
import random
import shutil
import numpy as np

from tensorflow.keras import layers
from tensorflow.python.ops import math_ops
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

ModuleNotFoundError: ignored

In [ ]:
git_dir = "/content/IOH-Chat-App"
git_url = "https://github.com/Bangkit-Capstone-Team/IOH-Chat-App.git"

if not os.path.exists(git_dir):
  subprocess.call(["git", "clone", git_url])

In [ ]:
def load_data(filedir):
  df = pd.read_csv(filedir)

  return df.English.tolist(), df.Indonesia.values.tolist()

In [ ]:
dataset_dir = "/content/IOH-Chat-App/Machine Learning/datasets/translate sentence/result/eng-ind.csv"

eng_sentences, ind_sentences = load_data(dataset_dir)

print(f"Num of english sentence: {len(eng_sentences)}")
print(f"Num of indonesia sentence: {len(ind_sentences)}")
print()
print(f"English example: {eng_sentences[-1]}")
print(f"Indonesia example: {ind_sentences[-1]}")

In [ ]:
def normalize_and_preprocess(text):
  text = tf_text.normalize_utf8(text).numpy().decode()
  text = text.lower().strip()
  text = text.replace("\t\n", "")

  return text

In [ ]:
eng_sentences = np.array(list(map(normalize_and_preprocess, eng_sentences)))
ind_sentences = np.array(list(map(normalize_and_preprocess, ind_sentences)))

In [ ]:
def tokenizer(sentence, max_vocab):
  tokenizer = Tokenizer(num_words=max_vocab)
  tokenizer.fit_on_texts(sentence)

  return tokenizer

In [ ]:
def pad_seqs(tokenizer, maxlen=None):
  return pad_sequences(tokenizer, maxlen=maxlen, padding="post", truncating="post")

In [ ]:
max_vocab = 8000

eng_tokenizer = tokenizer(eng_sentences, max_vocab)
ind_tokenizer = tokenizer(ind_sentences, max_vocab)

eng_tokenizer.fit_on_texts(eng_sentences)
ind_tokenizer.fit_on_texts(ind_sentences)

eng_encode_example = eng_tokenizer.texts_to_sequences(eng_sentences)
ind_encode_example = ind_tokenizer.texts_to_sequences(ind_sentences)

eng_vocab = eng_tokenizer.index_word
ind_vocab = ind_tokenizer.index_word

eng_decode_example = eng_tokenizer.sequences_to_texts(eng_encode_example)
ind_decode_example = ind_tokenizer.sequences_to_texts(ind_encode_example)

eng_maxlen = max([len(i)for i in eng_decode_example])
ind_maxlen = max([len(i)for i in ind_decode_example])

print(f"English sentence: {eng_decode_example[-1]}")
print(f"English sequences: {eng_encode_example[-1]}")
print()
print(f"Indonesia sentence: {ind_decode_example[-1]}")
print(f"Indonesia sequences: {ind_encode_example[-1]}")

In [ ]:
batch_size = 128
buffer_size = len(eng_sentences)

eng_pad_seqs = tf.convert_to_tensor(pad_seqs(eng_encode_example, maxlen=eng_maxlen))
ind_pad_seqs = tf.convert_to_tensor(pad_seqs(ind_encode_example, maxlen=eng_maxlen))

dataset = tf.data.Dataset.from_tensor_slices((eng_pad_seqs, ind_pad_seqs)).shuffle(buffer_size).batch(batch_size)

# eng_pad_seqs = tf.reshape(eng_pad_seqs, (*eng_pad_seqs.shape, 1))
# ind_pad_seqs = tf.reshape(ind_pad_seqs, (*eng_pad_seqs.shape[:-1], 1))

for input, target in dataset.take(1):
  print(f"English sentence: {eng_decode_example[-1]}")
  print(f"English sequences: {input}")
  print()
  print(f"Indonesia sentence: {ind_decode_example[-1]}")
  print(f"Indonesia sequences: {target}")

In [ ]:
embed_dims = 256
epochs = 5

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
       from_logits=True, reduction='none')

In [ ]:
def build_model(input_len, output_len, embed_dims, maxlen):
  # # Model Architecure 1
  # en_inputs = layers.Input(shape=(maxlen, 1))
  # en_embedding = layers.Embedding(input_len, embed_dims)(en_inputs)
  
  # en_lstm = layers.LSTM(512, return_state=True)
  # en_outputs, state_h, state_c = en_lstm(en_inputs)
  # en_states = [state_h, state_c]

  # dec_inputs = layers.Input(shape=(maxlen, 1))
  # dec_embedding = layers.Embedding(output_len, embed_dims)(dec_inputs)
  
  # dec_lstm = layers.LSTM(512, return_sequences=True, return_state=True)
  # dec_outputs, _, _ = dec_lstm(dec_inputs, initial_state=en_states)
  # x = layers.Dense(1024, activation=tf.nn.relu)(dec_outputs)
  # x = layers.Dropout(.5)(x)
  # outputs = layers.Dense(output_len, activation=tf.nn.softmax)(x)

  # model = Model([en_inputs, dec_inputs], outputs)

  # Model Architecure 2
  en_inputs = layers.Input(shape=(maxlen,))
  en_embedding = layers.Embedding(input_len, embed_dims)(en_inputs)
  
  en_gru = layers.GRU(512, return_sequences=True, return_state=True,)
  en_outputs, en_state = en_gru(en_embedding)

  dec_embedding = layers.Embedding(output_len, embed_dims)(en_inputs)
  
  dec_gru = layers.GRU(512, return_sequences=True, return_state=True)
  dec_outputs, dec_state = dec_gru(dec_embedding, initial_state=en_state)

  x = layers.Dense(512, activation=tf.nn.relu)(dec_outputs)
  x = layers.Dropout(.5)(x)
  x = layers.Dense(1024, activation=tf.nn.relu)(dec_outputs)
  x = layers.Dropout(.5)(x)
  outputs = layers.Dense(output_len, activation=tf.nn.softmax)(x)

  model = Model(en_inputs, outputs)

  # # Model Architecure 3
  # model = tf.keras.Sequential()
  
  # model.add(layers.Input(shape=(maxlen,)))
  # model.add(layers.Embedding(input_len, embed_dims))
  # model.add(layers.GRU(512, return_sequences=True, return_state=True))
  # model.add(layers.Embedding(output_len, embed_dims))
  # model.add(layers.GRU(512, return_sequences=True, return_state=True))  
  # model.add(layers.Dense(512, activation=tf.nn.relu))
  # model.add(layers.Dropout(.5))
  # model.add(layers.Dense(1024, activation=tf.nn.relu))
  # model.add(layers.Dropout(.5))
  # model.add(layers.Dense(output_len, activation=tf.nn.softmax))

  model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=["accuracy"]
  )

  return model

In [ ]:
checkpoint_path = "/content/IOH-Chat-App/Machine Learning/code/translate sentence/training_checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    save_weights_only=True,
    save_best_only=True,
    save_freq=10,
    verbose=1, 
)

model = build_model(
    len(eng_vocab) + 1, 
    len(ind_vocab) + 1, 
    embed_dims,
    eng_maxlen,
)

model.summary()

model.save_weights(checkpoint_path.format(epoch=0))

In [ ]:
model.fit(dataset,
          epochs=epochs,
          callbacks=[cp_callback],
          verbose=1)

In [2]:
saved_model_path = "/content/IOH-Chat-App/Machine Learning/code/translate sentence/saved_model/model.h5"
saved_model_dir = os.path.dirname(saved_model_path)

if os.path.exists(saved_model_dir):
  shutil.rmtree(saved_model_dir)

model.save(saved_model_path)

NameError: ignored

In [3]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

NameError: ignored

In [ ]:
def load_model(model_path):
  model = tf.keras.models.load_model(model_path)

  en_outputs, en_state = model.layers[2].output
  en_model = Model(model.input[0], en_state)

  dec_state = layers.Input(shape=(512,))
  dec_inputs = dec_state

  dec_gru = model.layers[3]
  dec_outputs, dec_state= dec_gru(model.input[0], initial_state=dec_inputs)
  
  dec_dense1 = model.layers[4](dec_outputs)
  dec_dropout1 = model.layers[5](dec_dense1)
  dec_dense2 = model.layers[6](dec_dropout1)
  dec_dropout2 = model.layers[7](dec_dense2)

  dec_model = Model(model.input[0] + dec_inputs, 
                    [dec_dropout2] + dec_state)
  
  return en_model, dec_model

In [ ]:
en_model, dec_model = load_model(saved_model_path)

In [ ]:
def translate(text):
  tokens = list()

  sequences = eng_tokenizer.texts_to_sequences([text])
  sequences = tf.convert_to_tensor(pad_seqs(sequences))

  input = en_model.predict(sequences)
  target_seq = np.zeros((1, 1))
  
  for i in sequences:
    output_chars, h, c = dec_model.predict([target_seq] + input)
    char_index = np.argmax(output_chars)
    text_char = ind_tokenizer.index_word[char_index]
    tokens.append(text_char)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = char_index
    states_value = [h, c]
  
  sentence = " ".join(tokens)
  return sentence

In [ ]:
translate("if a person has not had a chance to acquire his target language by the time")